# Tillegg 10.1: Kjedeprompting- [Lærdom](#lesson)- [Eksempel Lekeplass](#example-playground)## OppsettKjør følgende oppsett-celle to load your API-nøkkel og etablere `get_completion` hjelpefunksjon.

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Has been rewritten to take in a messages list of arbitrary length
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---## LærdomDe saying goes, "Writing is rewriting." It turns out, **Claude can often improve the accuracy of its response when asked to do so**!Dere are many ways to prompt Claude to "think again". De ways that feel natural to ask a human to double check their work will also generally work for Claude. (Check out our [prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts) for further examples of when and how to use prompt chaining.)

### EksempelsIn this example, we ask Claude to come up with ten words... but one or more of them isn't a real word.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

**Asking Claude to make its answer more accurate** fixes the error! Below, we've pulled down Claude's incorrect response from above and added another turn to the conversation asking Claude to fix its previous answer.

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

Men endrer Claude svaret sitt bare fordi vi sa det? Hva hvis vi starter med med et riktig svar allerede? Vil Claude miste selvtilliten? Here, we've placed et riktig svar i stedet for `first_response` og bedt den om å dobbeltsjekke igjen.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

Du kan merke at if you generate a respnse fra blokken over noen ganger, lar Claude ordene være som de er de fleste ganger, men endrer fortsatt av og til ordene selv om de alle allerede er riktige. Hva kan vi gjøre for å redusere dette? Per Kapittel 8, kan vi gi Claude en utvei! La oss prøve dette én gang til.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

Prøv å generere svar fra koden over noen ganger for å se at Claude er mye bedre på å holde seg til sine standpunkter nå.Du kan også bruke prompt chaining to **ask Claude to make its responses better**. Below, we asked Claude om først å skrive en historie, og deretter forbedre historien den skrev. Din personlige smak kan variere, men mange kan være enige om at Claude's andre versjon er bedre.First, let's generate Claude's første versjon av historien.

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Nå la oss få Claude til å forbedre førsteutkastet sitt.

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

Denne formen for substitusjon er veldig kraftig. Vi har brukt substitusjonsplassholdere til å sende inn lister, words, Claude's former responses, og så videre. You can also **use substitution til å gjøre det vi kaller "function calling," som er å be Claude om å utføre en funksjon, og deretter ta resultatene av den funksjonen og be Claude om å gjøre enda mer etterpå med resultatene**. Det fungerer som enhver annen substitusjon. Mer om dette i neste tillegg.Nedenfor er ett eksempel til på å ta resultatene av ett kall til Claude og plugge det inn i et annet, lengre kall. La oss starte med den første prompten (which includes prefilling Claude's svar denne gangen).

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

La oss sende denne listen med navn inn i en annen prompt.

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

Nå som du har lært om prompt chaining, gå til Tillegg 10.2 for å lære hvordan man implementerer function calling ved å bruke prompt chaining.

---## Eksempel LekeplassDette er et område for deg å eksperimentere fritt med prompten examples vist i denne leksjonen og justere prompts for å se hvordan det kan påvirke Claude's svars.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))